In [1]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
 
include("Data/Wikivote/Readwikivote.jl")
m = readdlm("Data/Wikivote/wiki-Vote.txt");

const dim0 = size(m);
M_int = Array{Int64,2}(dim0[1],2);
for i =1:dim0[1]
	for j=1:2
		M_int[i,j] = Int64(m[i,j]);
	end
end	

N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]);

A0 =spzeros(N,N);
A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N);
A0 = A0+A0';
A,p = largest_component(A0);


A0=0;M_int=0;p=0; m=0;
gc();

Read the file and construct the sparse affinity matrix 


In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[94.2059, 79.3881, 61.5265, 54.112, 44.636, 41.0007, 37.2999, 34.1442, 33.0396, 31.078, 28.5853, 28.433, 27.6056, 26.8602, 26.1597, 25.4311, 24.8522, 24.7663, 24.1516, 23.935, 23.4879, 23.1212, 22.7672, 22.4074, 21.957, 21.6821, 21.4898, 21.2703, 21.0723, 20.9768, 20.7423, 20.5539, 20.2083, 19.9561, 19.8485, 19.7501, 19.6136, 19.3887, 19.1173, 18.9497, 18.8365, 18.6478, 18.6819, 18.5944, 18.3822, 18.2273, 18.3876, 18.0426, 17.9748, 17.9839, 17.8478, 17.6668, 17.4034, 17.4731, 17.3502, 17.3749, 17.1529, 17.0527, 16.9455, 16.6663, 16.725, 16.8272, 16.6347, 16.4652, 16.4629, 16.5041, 16.3518, 16.3075, 16.1142, 16.0191, 16.1314, 15.9671, 15.9577, 15.9149, 15.7412, 15.7153, 15.665, 15.5971, 15.4996, 15.453, 15.4321, 15.2165, 15.178, 15.2527, 15.1733, 15.0947, 15.0747, 14.9057, 14.9597, 14.9271, 14.8679, 14.7071, 14.7873, 14.7255, 14.6661, 14.5356, 14.4049, 14.4605, 14.4394, 14.3866]

In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:100
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 1
Number of communities: 6
Modularity: 0.42285502781128015
n: 3
